<a href="https://colab.research.google.com/github/TrondVQ/FYS_STK_P2/blob/main/MethodVerification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures

In [41]:
#Verification of critical components in the implementations.

# Design matrix function:
def Design(x, p):
    """Create a design matrix for polynomial regression."""
    X = np.ones((x.shape[0], p + 1))  # Initialize matrix with ones
    for i in range(1, p + 1):  # Fill the matrix with powers of x
        X[:, i] = x[:, 0] ** i
    return X

# Test data
x_test = np.array([[1], [2], [3]])

# Polynomial degree
p_test = 2

# Custom Design Matrix
X_custom = Design(x_test, p_test)

# Scikit-learn PolynomialFeatures
poly = PolynomialFeatures(degree=p_test)
X_sklearn = poly.fit_transform(x_test)

# Compare the results
print("Custom Design Matrix:")
print(X_custom)

print("\nScikit-learn PolynomialFeatures Design Matrix:")
print(X_sklearn)

# Check if both results are the same
print("\nAre the results identical? ", np.allclose(X_custom, X_sklearn))

Custom Design Matrix:
[[1. 1. 1.]
 [1. 2. 4.]
 [1. 3. 9.]]

Scikit-learn PolynomialFeatures Design Matrix:
[[1. 1. 1.]
 [1. 2. 4.]
 [1. 3. 9.]]

Are the results identical?  True


In [42]:
# Activation functions and cost functions tests:
import numpy as np

# Sigmoid and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

# ReLU and its derivative
def ReLU(z):
    return np.where(z > 0, z, 0)

def ReLU_der(z):
    return np.where(z > 0, 1, 0)

# Leaky ReLU and its derivative
def leaky_ReLU(x, alpha=0.01):
    return np.where(x > 0, x, alpha * x)

def der_leaky_ReLU(x, alpha=0.01):
    return np.where(x > 0, 1, alpha)

# Mean Squared Error and its derivative
def mean_squared_error(predictions, targets):
    return np.mean((predictions - targets) ** 2)

def mean_squared_error_derivative(predictions, targets):
    return 2 * (predictions - targets) / targets.size

# Linear and its derivative
def linear(x):
    return x

def linear_derivative(x):
    return np.ones_like(x)

# R² score
def R2(z, zpred):
    return 1 - np.sum((z - zpred) ** 2) / np.sum((z - np.mean(z)) ** 2)

# Assessment Function to test each function with assert statements
def assess_functions():
    # 1. Sigmoid and its derivative
    x = np.array([0, 1, -1])
    assert np.allclose(sigmoid(x), [0.5, 0.73105858, 0.26894142]), f"Sigmoid failed for input {x}"
    assert np.allclose(sigmoid_derivative(x), [0.25, 0.19661193, 0.19661193]), f"Sigmoid derivative failed for input {x}"

    # 2. ReLU and its derivative
    z = np.array([0, 1, -1])
    assert np.allclose(ReLU(z), [0, 1, 0]), f"ReLU failed for input {z}"
    assert np.allclose(ReLU_der(z), [0, 1, 0]), f"ReLU derivative failed for input {z}"

    # 3. Leaky ReLU and its derivative
    x_leaky = np.array([0, 1, -1])
    assert np.allclose(leaky_ReLU(x_leaky, alpha=0.01), [0, 1, -0.01]), f"Leaky ReLU failed for input {x_leaky}"
    assert np.allclose(der_leaky_ReLU(x_leaky, alpha=0.01), [0.01, 1, 0.01]), f"Leaky ReLU derivative failed for input {x_leaky}"

    # 4. Mean Squared Error and its derivative
    predictions = np.array([0.5, 0.2, 0.9])
    targets = np.array([1, 0, 1])
    mse = mean_squared_error(predictions, targets)
    # Corrected expected MSE value
    assert np.isclose(mse, 0.10000000000000002), f"Mean Squared Error failed, got {mse}"

    mse_derivative = mean_squared_error_derivative(predictions, targets)
    # Corrected expected MSE derivative values
    assert np.allclose(mse_derivative, [-0.33333333, 0.13333333, -0.06666667]), f"Mean Squared Error Derivative failed, got {mse_derivative}"

    # 5. Linear activation and its derivative
    x_linear = np.array([1, 2, 3])
    assert np.allclose(linear(x_linear), [1, 2, 3]), f"Linear function failed for input {x_linear}"
    assert np.allclose(linear_derivative(x_linear), [1, 1, 1]), f"Linear derivative failed for input {x_linear}"

    # 6. R² score
    z_true = np.array([3, -0.5, 2, 7])
    z_pred = np.array([2.5, 0.0, 2, 8])
    r2 = R2(z_true, z_pred)
    assert np.isclose(r2, 0.9486081370449679), f"R² Score failed, got {r2}"

    print("All tests passed successfully!")

# Run assessments
assess_functions()




All tests passed successfully!


In [43]:
# Classes:
class NetworkClass:
    def __init__(self, cost_fun, cost_der, network_input_size, layer_output_sizes, activation_funcs, activation_ders):
        self.cost_fun = cost_fun
        self.cost_der = cost_der
        self.layers = []
        self.activation_funcs = activation_funcs
        self.activation_ders = activation_ders

        # Architecture check
        # print("Architecture configuration:")
        # print("--------------------------")
        # print("Network input size:", network_input_size)
        # print("Layer output sizes:", layer_output_sizes)
        # print("Activation functions:", activation_funcs)
        # print("Activation derivatives:", activation_ders)

        input_size = network_input_size
        for i, output_size in enumerate(layer_output_sizes):
            self.layers.append({
                'weights': np.random.randn(input_size, output_size) * np.sqrt(2. / input_size),
                'biases': np.zeros((1, output_size))
            })
            input_size = output_size

    def predict(self, inputs):
        activations, _, _ = self._feed_forward_saver(inputs)
        return activations[-1]  # Return the final activation

    def cost(self, inputs, targets):
        predictions = self.predict(inputs)
        return self.cost_fun(predictions, targets)

    def _feed_forward_saver(self, inputs):
        activations = [inputs]
        zs = []
        layer_inputs = []

        for layer in self.layers:
            layer_inputs.append(activations[-1])
            z = np.dot(activations[-1], layer['weights']) + layer['biases']
            a = self.activation_funcs[len(activations) - 1](z)
            zs.append(z)
            activations.append(a)

        return activations, zs, layer_inputs

    def compute_gradient(self, inputs, targets):
        activations, zs, layer_inputs = self._feed_forward_saver(inputs)
        layer_grads = []
        output = activations[-1]

        delta = self.cost_der(output, targets) * self.activation_ders[len(self.layers) - 1](zs[-1])

        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]
            prev_activation = layer_inputs[i]
            layer_grads.append({
                'weights': np.dot(prev_activation.T, delta),
                'biases': np.sum(delta, axis=0, keepdims=True)
            })
            if i > 0:
                delta = np.dot(delta, layer['weights'].T) * self.activation_ders[i - 1](zs[i - 1])

        layer_grads.reverse()
        return layer_grads

    def update_weights(self, layer_grads, learning_rate, lmbd):
        for i, layer in enumerate(self.layers):
            layer['weights'] -= learning_rate * (layer_grads[i]['weights'] + lmbd * layer['weights'])
            layer['biases'] -= learning_rate * layer_grads[i]['biases']

    def train(self, X, y, epochs, batch_size, learning_rate, lmbd):
        for epoch in range(epochs):
            # Shuffle data
            indices = np.arange(X.shape[0])
            np.random.shuffle(indices)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            # Mini-batch training
            for start in range(0, X.shape[0], batch_size):
                end = min(start + batch_size, X.shape[0])
                X_batch = X_shuffled[start:end]
                y_batch = y_shuffled[start:end]

                layer_grads = self.compute_gradient(X_batch, y_batch)
                self.update_weights(layer_grads, learning_rate, lmbd)

            # Optional: Print cost at the end of each epoch
            epoch_cost = self.cost(X, y)
            #print(f"Epoch {epoch + 1}/{epochs}, Cost: {epoch_cost}")



# Logistic regression class
class Logisticregr:
    def __init__(self, X_data, Y_data, eta=0.01, lmbd=0.0, iterations=100, nbatches=100, gamma=0.9):
        self.X_data = X_data
        self.Y_data = Y_data
        self.iterations = iterations
        self.eta = eta
        self.lmbd = lmbd
        self.nbatches = nbatches
        self.gamma = gamma
        self.beta_l = None

    def sigmoid(self, X):
        return 1 / (1 + np.exp(-X))

    def predict(self, X):
        return self.sigmoid(X @ self.beta_l)

    def predict_class(self, X, threshold=0.5):
        return self.predict(X) >= threshold

    def accuracy(self, X, y):
        y_pred = self.predict_class(X)
        return np.mean(y_pred == y)

    # Stochastic gradient descent from a) with some modifications.
    def SGD(self, betainit=None):
        if betainit is None:
            betainit = np.random.rand(self.X_data.shape[1]) * 0.01  # Small random initialization

        beta = np.copy(betainit)
        betaold = np.zeros_like(beta)
        v = np.zeros_like(beta)

        print("Initial beta:", beta)

        ind = np.arange(self.X_data.shape[0])
        np.random.shuffle(ind)
        batch = np.array_split(ind, min(self.nbatches, len(ind)))

        for epoch in range(self.iterations):


            for k in range(len(batch)):
                betaold = np.copy(beta)

                xk = self.X_data[batch[k]]
                yk = self.Y_data[batch[k]]

                M = len(yk)

                predictions = self.sigmoid(xk @ beta)
                g = (1.0 / M) * xk.T @ (predictions - yk) + 2 * self.lmbd * beta
                #print("Gradient g:", g)
                v = self.gamma * v + self.eta * g
                beta -= v

            #print("Updated beta:", beta)  # Debug print for beta updates

        print(f"Stopped after {self.iterations} epochs")
        self.beta_l = beta
        return beta

In [49]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

# -------------------- Neural Network Comparison --------------------

# 1. Generate synthetic data for classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_classes=2, random_state=42)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Custom NetworkClass (MLP) model
# Define sigmoid and its derivative functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

# Initialize custom network
network = NetworkClass(cost_fun=mean_squared_error, cost_der=mean_squared_error_derivative,
                       network_input_size=X_train.shape[1],
                       layer_output_sizes=[10, 5, 1],
                       activation_funcs=[sigmoid, sigmoid, sigmoid],
                       activation_ders=[sigmoid_derivative, sigmoid_derivative, sigmoid_derivative])

y_train = y_train.reshape(-1, 1)
# Train custom neural network
network.train(X_train, y_train, epochs=100, batch_size=32, learning_rate=0.01, lmbd=0.01)
y_train = y_train.ravel()
# 3. Train a Scikit-learn MLPClassifier
mlp = MLPClassifier(learning_rate_init=0.01, solver = 'sgd', alpha= 0.01, batch_size=32, hidden_layer_sizes=(10, 5), activation='logistic', max_iter=100, random_state=42)
mlp.fit(X_train, y_train)

# 4. Evaluate both models
y_pred_network = network.predict(X_test)  # Custom network prediction
y_pred_network = (y_pred_network > 0.5).astype(int)  # Convert output to binary class

y_pred_mlp = mlp.predict(X_test)

# Accuracy comparison
accuracy_network = accuracy_score(y_test, y_pred_network)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

print(f"Custom Network Accuracy: {accuracy_network}")
print(f"MLPClassifier Accuracy: {accuracy_mlp}")


# -------------------- Logistic Regression Comparison Using SGDClassifier --------------------

# 1. Generate synthetic data for binary classification
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Custom Logistic Regression
log_reg_custom = Logisticregr(X_train, y_train, eta=0.01, lmbd=0.01, iterations=100, nbatches=10)
beta_custom = log_reg_custom.SGD()

# 3. Scikit-learn SGDClassifier (fixed to 'log_loss')
sgd_log_reg = SGDClassifier(loss='log_loss', max_iter=100, learning_rate='constant', eta0=0.01, alpha=0.01, random_state=42)
sgd_log_reg.fit(X_train, y_train)

# 4. Evaluate both models
y_pred_custom = log_reg_custom.predict_class(X_test)
y_pred_sgd_sklearn = sgd_log_reg.predict(X_test)

# Accuracy comparison
accuracy_custom_log_reg = accuracy_score(y_test, y_pred_custom)
accuracy_sgd_log_reg = accuracy_score(y_test, y_pred_sgd_sklearn)

print(f"Custom Logistic Regression Accuracy: {accuracy_custom_log_reg}")
print(f"SGDClassifier Logistic Regression Accuracy: {accuracy_sgd_log_reg}")


Custom Network Accuracy: 0.73
MLPClassifier Accuracy: 0.855
Initial beta: [0.00634343 0.00634943 0.0094459  0.00474511 0.00712479 0.00688394
 0.00756132 0.00747431 0.00185482 0.00351333 0.00145248 0.00526929
 0.00646167 0.00874306 0.00999033 0.00656449 0.00497204 0.00732771
 0.00055631 0.00986166]
Stopped after 100 epochs
Custom Logistic Regression Accuracy: 0.78
SGDClassifier Logistic Regression Accuracy: 0.795


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
